# Lesson 4: Creating an MCP Server

이 레슨에서는 이전 레슨의 chatbot tool들을 래핑하여 2개의 tool을 노출하는 MCP server를 구축할 것입니다. 여기서는 `stdio` transport를 사용하고 제공된 로컬 environment에서 server를 실행할 것입니다. 원격 server에 대해서는 다른 레슨에서 더 자세히 배울 예정입니다.

## How can you create an MCP Server? - *Additional Note*

MCP server를 생성하는 방법은 두 가지가 있습니다:
- **low-level implementation**: 이 접근법에서는 다양한 유형의 request들(`ListToolsRequest`와 `CallToolRequest`)을 직접 정의하고 처리합니다. 이 접근법을 통해 server의 모든 측면을 커스터마이징할 수 있습니다.
- **`FastMCP`를 사용한 high-level implementation**: `FastMCP`는 MCP server 구축을 더 빠르고 간단하게 만드는 high-level interface입니다. 이 접근법에서는 tool들을 function으로 정의하는 것에만 집중하면 되고, `FastMCP`가 모든 protocol 세부사항을 처리합니다.

이 레슨에서는 `FastMCP`를 사용할 것입니다. low-level 접근법에 대해 더 자세히 알고 싶다면, 이 notebook 마지막에 있는 resource들을 확인해 보세요.

## Building your MCP Server using `FastMCP`

MCP server에 필요한 file들을 `mcp_project` folder에 구축할 것입니다. 다음 레슨들에서 이 project folder에 더 많은 file들을 점진적으로 추가하거나 업데이트할 예정입니다. `mcp_project` folder는 제공되어 있습니다. 먼저 server의 python file인 `research_server.py`를 생성하여 `mcp_project` folder에 저장한 다음, server를 실행할 environment를 준비할 것입니다.

`FastMCP`를 사용하여 MCP server를 생성하려면, `mcp`라는 label로 `FastMCP` server를 초기화하고 function들을 `@mcp.tool()`로 decorating해야 합니다. `FastMCP`는 type hint와 docstring을 기반으로 필요한 MCP schema를 자동으로 생성합니다.


**참고**: magic function `%%writefile mcp_project/research_server.py`는 code를 실행하지 않고 cell의 내용을 `mcp_project` directory의 server file `research_server.py`에 저장합니다. magic function을 제거하고 cell을 실행하면, code는 Jupyter notebook에서 실행되지 않습니다. 다음 section에서 terminal에서 server를 실행할 것입니다.

In [1]:
import os
os.makedirs('mcp_project', exist_ok=True)

In [2]:
%%writefile mcp_project/research_server.py

import arxiv
import json
import os
from typing import List
from mcp.server.fastmcp import FastMCP


PAPER_DIR = "papers"

# Initialize FastMCP server
mcp = FastMCP("research")

@mcp.tool()
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """
    
    # Use arxiv to find the papers 
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

@mcp.tool()
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."



if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Writing mcp_project/research_server.py


## Setting up your Environment & Testing your Server

이제 server를 실행하고 테스트하는 데 사용할 environment를 설정할 것입니다. 이를 위해 Python environment 관리를 도와주는 `uv` tool을 사용할 것입니다. 이 tool은 project file들을 자동으로 설정하고 package dependency들을 관리합니다.

**Terminal Instructions**

In [ ]:
npx @modelcontextprotocol/inspector python3 research_server.py

To run the MCP server locally:
1. Open your system terminal
2. Navigate to the mcp_project directory: cd mcp_project
3. Create virtual environment: uv venv
4. Activate virtual environment: source .venv/bin/activate
5. Install dependencies: uv add mcp arxiv
6. Launch inspector: npx @modelcontextprotocol/inspector uv run research_server.py
